# IMAGE SUPER RESOLUTION
This notebook can be used to upscale (enhance) low resolution images.
For example, this can be used to upscale images created with VQGAN+STEP.

Follow the inscructions below to execute the notebook.


# You can use a CPU instance. No need for GPU!

This notebook is based on https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Prediction_Tutorial.ipynb [![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Prediction_Tutorial.ipynb)

Special thanks to: 
https://idealo.github.io/image-super-resolution/ for providing ISR
and thanks to Google Colab for providing a place to run this notebook.


In [ ]:
!pip install 'h5py==2.10.0' --force-reinstall

# Restart the runtime with the button that is displayed above and continue with the next cell after restart.

In [ ]:
!pip3 install ISR

In [ ]:
# @markdown # Mount your google drive! ... or not

mount_gdrive = True #@param {type:"boolean"}
if mount_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')


In [ ]:
# @markdown # Select the model(s) for ISR
# @markdown if more than one model is selected, all of the selected models will be downloaded and used


import numpy as np
from PIL import Image
import os
from ISR.models import RDN, RRDN

noise_cancel = True #@param {type:"boolean"}
gans = False #@param {type:"boolean"}
psnr_small = False #@param {type:"boolean"}
psnr_large = False #@param {type:"boolean"}

models = { }
if noise_cancel:
  models['noise-cancel'] = RDN(weights='noise-cancel')
if gans:
  models['gans'] = RRDN(weights='gans')
if psnr_small:
  models['psnr-small'] = RDN(weights='psnr-small')
if psnr_large:
  models['psnr-large'] = RDN(weights='psnr-large')


In [ ]:
!ls '/content/drive/MyDrive/ISR-output/' | wc -l

In [ ]:
# @markdown # Enhance !!1!

# @markdown ## select input and output folders

input_directory = '/content/drive/MyDrive/ISR-input/' #@param {type:"string"}
output_directory = '/content/drive/MyDrive/ISR-output/' #@param {type:"string"}
!mkdir -p {input_directory}
!mkdir -p {output_directory}


imagecounter = 0

# @markdown ## select one or more enhancements. 
enhancements = 2 #@param {type:"slider", min:1, max:5, step:1}
# @markdown Careful, the image size is quadrupled with each enhancement.

print("number of files in input folder:")
!ls -l {input_directory} | wc -l

for model_name in models.keys():
  print("model: " + model_name)
  for filename in os.listdir(input_directory):
      if filename.endswith(".jpg") or filename.endswith(".png"):

          if  imagecounter < 2598: # for skipping already processed images, when colab kicks you out
            imagecounter+=1
            continue

          print("input: " + os.path.join(input_directory, filename))
          imagepath = (os.path.join(input_directory, filename))
          image = Image.open(imagepath)

          for enhancement in range(enhancements):
            print("enhancement:" + str(enhancement+1) + " of " + str(enhancements))
            sr_img = models[model_name].predict(np.array(image))
            image = Image.fromarray(sr_img)
          
          output_filename = (os.path.join(output_directory, filename))  + "_" + model_name + "_x" + str(enhancements) + "_ISR.jpg"
          print("output: " + output_filename)
          image.save(output_filename)

          imagecounter+=1
          print(imagecounter)


# @markdown # Do not forget to move your input images to the input folder!

